In [1]:
!pip install gpxpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 998.6 kB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import json
import os
import gpxpy
from datetime import datetime
from dateutil import parser

In [3]:
workouts_folder_path1 = "/content/drive/MyDrive/Dane z aplikacji/Endomondo/Workouts"
workouts_folder_path2 = "/content/drive/MyDrive/Dane z aplikacji/MapMyRun/user184238914_workout_history.csv"
workouts_folder_path3 = "/content/drive/MyDrive/Dane z aplikacji/Sports Tracker/Running workouts"
workouts_folder_path4 = "/content/drive/MyDrive/Dane z aplikacji/Garmin Connect/Aktywności/Bieganie.csv"

**df1 => Endomondo export**

In [4]:
dfs = []

files = [f for f in os.listdir(workouts_folder_path1) if f.endswith(".json")]

if files:
    for file_name in files:
        file_path = os.path.join(workouts_folder_path1, file_name)

        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                if isinstance(data, list) and len(data) > 0:
                    sport = 'Running' if data[0].get('sport') == 'RUNNING' else data[0].get('sport')
                    if sport == 'Running':
                        source = data[1].get('source')
                        source = source.replace("INPUT_MANUAL", "Endomondo (Manual)").replace("TRACK_MOBILE", "Endomondo (GPS)")

                        start_time = pd.to_datetime(data[3].get('start_time'))
                        end_time = pd.to_datetime(data[4].get('end_time'))
                        speed_avg_kmh = round(float(data[8].get('speed_avg_kmh')), 2) if data[8].get('speed_avg_kmh') is not None else None
                        duration = (str(data[5].get('duration_s')).split('.')[0])if data[5].get('duration_s') is not None else None
                        distance = round(float(data[6].get('distance_km')), 2) if duration is not None else None

                        dfs.append({
                            'sport': sport,
                            'source': source,
                            'date': start_time.date(),
                            'start_time': start_time.time(),
                            'end_time': end_time.time(),
                            'duration_s': duration,
                            'distance_km': distance,
                            'speed_avg_kmh': speed_avg_kmh
                        })

            except Exception as e:
                print(f"Error processing JSON file {file_name}: {e}")

    df1 = pd.DataFrame(dfs).sort_values(by='date').reset_index(drop=True)
    df1['date'] = df1['date'].astype(str).str.strip()
    df1['date'] = pd.to_datetime(df1['date'])
    df1['distance_km'] = df1['distance_km'].astype(float).round(2)
    df1['duration_s'] = pd.to_numeric(df1['duration_s'], errors='coerce').astype('Int64')
    display(df1)
else:
    print("No JSON files found in the specified folder.")


,sport,source,date,start_time,end_time,duration_s,distance_km,speed_avg_kmh
0,Running,Endomondo (Manual),2012-08-15,10:07:00,10:33:00,1560,4.21,9.72
1,Running,Endomondo (Manual),2012-08-18,18:07:00,19:04:00,3420,8.48,8.93
2,Running,Endomondo (Manual),2012-08-20,18:30:00,19:19:00,2940,6.73,8.24
3,Running,Endomondo (Manual),2012-08-22,18:30:00,19:24:00,3240,7.77,8.63
4,Running,Endomondo (Manual),2012-08-26,18:21:00,19:13:00,3120,7.77,8.97
...,...,...,...,...,...,...,...,...
646,Running,Endomondo (GPS),2020-10-25,05:47:17,06:46:08,3522,10.07,NaN
647,Running,Endomondo (GPS),2020-10-29,14:59:15,16:00:49,3673,10.03,NaN
648,Running,Endomondo (GPS),2020-11-01,16:52:07,16:52:10,3239,9.44,NaN
649,Running,Endomondo (GPS),2020-11-04,19:53:52,19:53:55,3370,9.37,NaN


**df2 => Map My Run export**

In [6]:
df = pd.read_csv(workouts_folder_path2, header=None, skip_blank_lines=False)

df2 = df[df[2] == 'Run'].copy()

df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source2', 14:'url'}, inplace=True)
df2['sport'] = df2['sport'].apply(lambda x: 'Running' if x == 'Run' else x)
df2['source'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

df2['date'] = df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
df2['date'] = pd.to_datetime(df2['date'])
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['duration_s'] = df2['duration_s'].astype(int)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)

df2 = df2[['sport', 'source', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]

display(df2)


,sport,source,date,duration_s,distance_km,speed_avg_kmh
1,Running,Map My Run app,2020-12-17,600,1.61,9.66
3,Running,Map My Run app,2020-11-08,3163,9.16,10.43
4,Running,Map My Run app,2020-11-04,3370,9.37,10.01
7,Running,Map My Run app,2020-11-01,3239,9.44,10.49
11,Running,Map My Run app,2020-10-29,3673,10.03,9.83
...,...,...,...,...,...,...
4776,Running,Map My Run app,2012-08-26,3120,7.77,8.97
4777,Running,Map My Run app,2012-08-22,3240,7.77,8.63
4778,Running,Map My Run app,2012-08-20,2940,6.73,8.24
4779,Running,Map My Run app,2012-08-18,3420,8.48,8.93


**df3 => export from Sports Tracker app**

In [8]:
gpx_data_list = []

for filename in os.listdir(workouts_folder_path3):
    if filename.endswith(".gpx"):
        gpx_file_path = os.path.join(workouts_folder_path3, filename)

        with open(gpx_file_path, "r") as gpx_file:
            gpx_data = gpxpy.parse(gpx_file)

            if gpx_data.tracks and gpx_data.tracks[0].segments and gpx_data.tracks[0].segments[0].points:
                start_time = gpx_data.tracks[0].segments[0].points[0].time

                date = start_time.strftime("%Y-%m-%d")
                start_time = start_time.strftime("%H:%M:%S")
                end_time = gpx_data.tracks[0].segments[0].points[-1].time
                duration_seconds = int((end_time - gpx_data.tracks[0].segments[0].points[0].time).total_seconds())

                distance_km = round((gpx_data.length_3d() / 1000), 2)
                speed_avg_kmh = round(distance_km / (duration_seconds / 3600), 2)

                gpx_data_list.append({
                    "sport" : "Running",
                    "source" : "Sports Tracker",
                    "date": date,
                    "start_time": start_time,
                    "duration_s": duration_seconds,
                    "distance_km": distance_km,
                    "speed_avg_kmh": speed_avg_kmh
                })

df3 = pd.DataFrame(gpx_data_list)
df3['date'] = pd.to_datetime(df3['date'])

display(df3)


,sport,source,date,start_time,duration_s,distance_km,speed_avg_kmh
0,Running,Sports Tracker,2021-01-28,17:53:47,3392,10.09,10.71
1,Running,Sports Tracker,2021-01-23,10:51:01,3421,10.15,10.68
2,Running,Sports Tracker,2021-01-15,16:43:11,848,2.31,9.81
3,Running,Sports Tracker,2021-01-10,16:34:57,3651,10.11,9.97
4,Running,Sports Tracker,2020-12-27,07:58:19,3568,10.11,10.20
5,Running,Sports Tracker,2020-12-31,09:12:35,5369,15.32,10.27
6,Running,Sports Tracker,2020-12-23,11:28:26,4501,12.19,9.75
7,Running,Sports Tracker,2020-12-21,16:34:33,3170,9.32,10.58
8,Running,Sports Tracker,2020-12-19,08:05:39,2567,6.89,9.66
9,Running,Sports Tracker,2020-12-01,19:06:07,3538,10.25,10.43


**df4 => export from Garmin Connect till the end of 2023**

In [18]:
df4 = pd.read_csv(workouts_folder_path4, skip_blank_lines=False)
df4['source'] = 'Garmin Connect'
df4['Typ aktywności'] = df4['Typ aktywności'].replace('Bieganie', 'Running')
df4['Czas'] = pd.to_timedelta(df4['Czas']).dt.total_seconds()
df4['Czas'] = df4['Czas'].astype('int64')

df4['Data'] = pd.to_datetime(df4['Data'])
df4['start_time'] = df4['Data'].dt.time
df4['Data'] = df4['Data'].dt.date

#df4["Średnie tempo"] = df4["Średnie tempo"] * 60

df4.rename(columns={'Typ aktywności': 'sport', "Data": 'date', "Czas": "duration_s", "Dystans": "distance_km", "Średnie tempo": "speed_avg_kmh"}, inplace=True)

df4 = df4[['sport', 'source', 'date', 'start_time', 'duration_s', 'distance_km', 'speed_avg_kmh']]
display(df4)

,sport,source,date,start_time,duration_s,distance_km,speed_avg_kmh
0,Running,Garmin Connect,2023-11-05,09:38:17,2029,5.65,5:59
1,Running,Garmin Connect,2023-10-01,12:21:42,3002,7.37,6:47
2,Running,Garmin Connect,2023-09-05,19:44:07,3133,8.06,6:29
3,Running,Garmin Connect,2023-09-03,18:43:23,584,1.40,6:58
4,Running,Garmin Connect,2023-09-03,18:16:03,1276,4.02,5:18
...,...,...,...,...,...,...,...
146,Running,Garmin Connect,2017-12-02,12:28:00,3515,12.50,4:41
147,Running,Garmin Connect,2016-11-11,15:12:00,2808,10.00,4:41
148,Running,Garmin Connect,2016-08-26,11:15:00,1524,5.80,4:23
149,Running,Garmin Connect,2016-06-04,12:00:00,2767,10.00,4:37


In [ ]:
df = pd.read_csv(workouts_folder_path2, header=None, skip_blank_lines=False)

df2 = df[df[2] == 'Run'].copy()

df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source2', 14:'url'}, inplace=True)
df2['sport'] = df2['sport'].apply(lambda x: 'Running' if x == 'Run' else x)
df2['source2'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

df2['date'] = df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
df2['date'] = pd.to_datetime(df2['date'])
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['duration_s'] = df2['duration_s'].astype(int)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)

df2 = df2[['sport', 'source2', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]

display(df2)


,sport,source2,date,duration_s,distance_km,speed_avg_kmh
1,Running,Map My Run app,2020-12-17,600,1.61,9.66
3,Running,Map My Run app,2020-11-08,3163,9.16,10.43
4,Running,Map My Run app,2020-11-04,3370,9.37,10.01
7,Running,Map My Run app,2020-11-01,3239,9.44,10.49
11,Running,Map My Run app,2020-10-29,3673,10.03,9.83
...,...,...,...,...,...,...
4776,Running,Map My Run app,2012-08-26,3120,7.77,8.97
4777,Running,Map My Run app,2012-08-22,3240,7.77,8.63
4778,Running,Map My Run app,2012-08-20,2940,6.73,8.24
4779,Running,Map My Run app,2012-08-18,3420,8.48,8.93


In [ ]:
df = pd.read_csv(workouts_folder_path2, header=None, skip_blank_lines=False)

df2 = df[df[2] == 'Run'].copy()

df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source2', 14:'url'}, inplace=True)
df2['sport'] = df2['sport'].apply(lambda x: 'Running' if x == 'Run' else x)
df2['source2'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

df2['date'] = df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
df2['date'] = pd.to_datetime(df2['date'])
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['duration_s'] = df2['duration_s'].astype(int)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)

df2 = df2[['sport', 'source2', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]

display(df2)


,sport,source2,date,duration_s,distance_km,speed_avg_kmh
1,Running,Map My Run app,2020-12-17,600,1.61,9.66
3,Running,Map My Run app,2020-11-08,3163,9.16,10.43
4,Running,Map My Run app,2020-11-04,3370,9.37,10.01
7,Running,Map My Run app,2020-11-01,3239,9.44,10.49
11,Running,Map My Run app,2020-10-29,3673,10.03,9.83
...,...,...,...,...,...,...
4776,Running,Map My Run app,2012-08-26,3120,7.77,8.97
4777,Running,Map My Run app,2012-08-22,3240,7.77,8.63
4778,Running,Map My Run app,2012-08-20,2940,6.73,8.24
4779,Running,Map My Run app,2012-08-18,3420,8.48,8.93


In [ ]:
df = pd.read_csv(workouts_folder_path2, header=None, skip_blank_lines=False)

df2 = df[df[2] == 'Run'].copy()

df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source2', 14:'url'}, inplace=True)
df2['sport'] = df2['sport'].apply(lambda x: 'Running' if x == 'Run' else x)
df2['source2'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

df2['date'] = df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
df2['date'] = pd.to_datetime(df2['date'])
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['duration_s'] = df2['duration_s'].astype(int)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)

df2 = df2[['sport', 'source2', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]

display(df2)


,sport,source2,date,duration_s,distance_km,speed_avg_kmh
1,Running,Map My Run app,2020-12-17,600,1.61,9.66
3,Running,Map My Run app,2020-11-08,3163,9.16,10.43
4,Running,Map My Run app,2020-11-04,3370,9.37,10.01
7,Running,Map My Run app,2020-11-01,3239,9.44,10.49
11,Running,Map My Run app,2020-10-29,3673,10.03,9.83
...,...,...,...,...,...,...
4776,Running,Map My Run app,2012-08-26,3120,7.77,8.97
4777,Running,Map My Run app,2012-08-22,3240,7.77,8.63
4778,Running,Map My Run app,2012-08-20,2940,6.73,8.24
4779,Running,Map My Run app,2012-08-18,3420,8.48,8.93


In [ ]:
print(df1.dtypes)
print(df2.dtypes)
print(df3.dtypes)
print(df4.dtypes)

sport                    object
source1                  object
date             datetime64[ns]
start_time               object
end_time                 object
duration_s                Int64
distance_km             float64
speed_avg_kmh           float64
dtype: object
sport                    object
source2                  object
date             datetime64[ns]
duration_s                int64
distance_km             float64
speed_avg_kmh           float64
dtype: object
source3                  object
date             datetime64[ns]
start_time               object
duration_s                int64
distance_km             float64
speed_avg_kmh           float64
dtype: object
sport             object
source            object
date              object
start_time        object
duration_s        object
duration_s         int64
distance_km      float64
speed_avg_kmh     object
dtype: object


In [ ]:
merged_df = pd.merge(df2, df1, on=['date', 'distance_km'], how='left')
df12 = pd.DataFrame()

df12['sport'] = merged_df['sport_x']
df12['source1'] = merged_df['source1']
df12['source2'] = merged_df['source2']
df12['date'] = merged_df['date']
df12['start_time'] = merged_df['start_time']
df12['end_time'] = merged_df['end_time']
df12['distance_km'] = merged_df['distance_km']
df12['duration'] = merged_df['duration_s_x'].combine_first(merged_df['duration_s_y'])
df12['speed_avg_kmh'] = merged_df['speed_avg_kmh_x'].combine_first(merged_df['speed_avg_kmh_y'])

df12['info'] = ''

#mask_duration_not_nan = ~merged_df['duration_s_x'].isna() & ~merged_df['duration_s_y'].isna()
#mask_speed_not_nan = ~merged_df['speed_avg_kmh_x'].isna() & ~merged_df['speed_avg_kmh_y'].isna()

#result_df.loc[mask_duration_not_nan & (merged_df['duration_s_x'] != merged_df['duration_s_y']), 'info'] += 'duration mismatch; '
#result_df.loc[mask_speed_not_nan & (merged_df['speed_avg_kmh_x'] != merged_df['speed_avg_kmh_y']), 'info'] += 'speed_avg_kmh mismatch; '
# Add more conditions for other columns if needed

# Display the result
display(df12)
display(df12.dtypes)



,sport,source1,source2,date,start_time,end_time,distance_km,duration,speed_avg_kmh,info
0,Running,NaN,Map My Run app,2020-12-17,NaN,NaN,1.61,600,9.66,
1,Running,Endomondo (Manual),Map My Run app,2020-11-08,17:29:00,17:29:03,9.16,3163,10.43,
2,Running,Endomondo (GPS),Map My Run app,2020-11-04,19:53:52,19:53:55,9.37,3370,10.01,
3,Running,Endomondo (GPS),Map My Run app,2020-11-01,16:52:07,16:52:10,9.44,3239,10.49,
4,Running,Endomondo (GPS),Map My Run app,2020-10-29,14:59:15,16:00:49,10.03,3673,9.83,
...,...,...,...,...,...,...,...,...,...,...
713,Running,Endomondo (Manual),Map My Run app,2012-08-26,18:21:00,19:13:00,7.77,3120,8.97,
714,Running,Endomondo (Manual),Map My Run app,2012-08-22,18:30:00,19:24:00,7.77,3240,8.63,
715,Running,Endomondo (Manual),Map My Run app,2012-08-20,18:30:00,19:19:00,6.73,2940,8.24,
716,Running,Endomondo (Manual),Map My Run app,2012-08-18,18:07:00,19:04:00,8.48,3420,8.93,


sport                    object
source1                  object
source2                  object
date             datetime64[ns]
start_time               object
end_time                 object
distance_km             float64
duration                  int64
speed_avg_kmh           float64
info                     object
dtype: object

In [ ]:
df123 = pd.concat([df12, df3], ignore_index=True)
df123 = df123[['sport', 'source1', 'source2', 'source3', 'date', 'start_time', 'end_time', 'distance_km', 'duration', 'info', 'duration_s']]
df123.loc[df123['source3'] == 'Sports Tracker', 'duration'] = df123['duration_s']
df123 = df123.drop('duration_s', axis=1)

df123 = df123.sort_values(by='date')
display(df123)

,sport,source1,source2,source3,date,start_time,end_time,distance_km,duration,info
717,Running,Endomondo (Manual),Map My Run app,NaN,2012-08-15,10:07:00,10:33:00,4.21,1560.0,
716,Running,Endomondo (Manual),Map My Run app,NaN,2012-08-18,18:07:00,19:04:00,8.48,3420.0,
715,Running,Endomondo (Manual),Map My Run app,NaN,2012-08-20,18:30:00,19:19:00,6.73,2940.0,
714,Running,Endomondo (Manual),Map My Run app,NaN,2012-08-22,18:30:00,19:24:00,7.77,3240.0,
713,Running,Endomondo (Manual),Map My Run app,NaN,2012-08-26,18:21:00,19:13:00,7.77,3120.0,
...,...,...,...,...,...,...,...,...,...,...
723,NaN,NaN,NaN,Sports Tracker,2020-12-31,09:12:35,NaN,15.32,5369.0,NaN
721,NaN,NaN,NaN,Sports Tracker,2021-01-10,16:34:57,NaN,10.11,3651.0,NaN
720,NaN,NaN,NaN,Sports Tracker,2021-01-15,16:43:11,NaN,2.31,848.0,NaN
719,NaN,NaN,NaN,Sports Tracker,2021-01-23,10:51:01,NaN,10.15,3421.0,NaN


**Checks**

In [ ]:
df2['year_month'] = df2['date'].dt.to_period('M')


df3 = df2.groupby('year_month')['distance_km'].sum().reset_index()
df3['year_month'] = df3['year_month'].dt.to_timestamp().dt.strftime('%Y-%m')
display(df3)

In [ ]:
import pandas as pd
import numpy as np

google_sheets_link = 'https://docs.google.com/spreadsheets/d/1sKQRBWY5xvY-erhFxXo0bC0pTpZT6QUUMx1fjLngGk0/edit?usp=drive_link'

document_key = google_sheets_link.split('/')[-2]

worksheet_link = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv'

df4 = pd.read_csv(worksheet_link, header=1)
df4 = df4.drop(columns=['Unnamed: 0'])
df4 = df4.iloc[:, :-1]
df4 = df4.iloc[:-1,:]
df4.replace('-', np.nan, inplace=True)
display(df4)


In [ ]:
df5 = pd.DataFrame(columns=['year_month', 'distance_km'])

for index, row in df4.iterrows():
    for col in df4.columns[1:]:
        year_month = f'{int(col)}-{index + 1:02d}'
        distance_km = row[col]
        df5 = pd.concat([df5, pd.DataFrame({'year_month': [year_month], 'distance_km': [distance_km]})], ignore_index=True)

df5 = df5.sort_values(by='year_month')
df5 = df5[df5['year_month'].apply(lambda x: int(x.split('-')[1])) != 13]

df5['distance_km'] = df5['distance_km'].str.replace(',', '.').astype(float)
df5['distance_km'] = pd.to_numeric(df5['distance_km'])

display(df5)

In [ ]:
import pandas as pd

df3 = df3.rename(columns={'distance_km': 'df3_distance_km'})
df5 = df5.rename(columns={'distance_km': 'df5_distance_km'})

df6 = pd.concat([df3.set_index('year_month'), df5.set_index('year_month')], axis=1, join='outer')

df6['absolute_difference'] = abs(df6['df3_distance_km'] - df6['df5_distance_km'])

print(df6)

In [ ]:
filtered_df = df6[df6['absolute_difference'] >= 0.02].copy()
print(filtered_df)